# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [1]:
debugAsGym = False
testResult = False

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'A3C',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m in MR2, #3 is 0-400m in Supra, #4 is test ring
}

In [2]:
if debugAsGym:
    env = gymnasium.make("real-time-gym-v1", config=my_config)

In [3]:
if debugAsGym:
    env.reset()

# Register the environment in a way that RLlib is happy

In [4]:
if not debugAsGym and not testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [5]:
if not debugAsGym and not testResult:
    import ray
    ray.shutdown()
    ray.init()

2023-05-24 08:28:10,106	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [6]:
if not debugAsGym and not testResult:
    from ray.rllib.algorithms.sac import SACConfig
    from ray.rllib.algorithms.ppo import PPOConfig
    from ray.rllib.algorithms.a2c import A2CConfig

    algo = (
        A2CConfig()
        .resources(
            num_gpus_per_learner_worker=1,
            #num_cpus_for_local_worker=1,
            num_cpus_per_learner_worker=16
            )
        .rollouts(
            num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="truncate_episodes",
            #rollout_fragment_length=256
            )
        .framework(
            framework="torch",
            #eager_tracing=True,
            )
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=155,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [1, 8]},
        )
        .build()
    )

2023-05-24 08:28:19,216	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=15280) GT Real Time instantiated
(RolloutWorker pid=15280) GT AI Server instantiated for rtgym
(RolloutWorker pid=15280) still simple reward system
(RolloutWorker pid=15280) starting up on localhost port 9999
(RolloutWorker pid=15280) Waiting for a connection


2023-05-24 08:29:54,772	INFO trainable.py:172 -- Trainable.setup took 95.556 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(RolloutWorker pid=15280) Connection from ('127.0.0.1', 50186)


In [7]:
if not debugAsGym and not testResult:
    N = 2000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 10 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()


(RolloutWorker pid=15280) reset triggered
(RolloutWorker pid=15280) reload save for track : 4
Loop:  0
Saved 0
(RolloutWorker pid=15280) reset triggered
(RolloutWorker pid=15280) reload save for track : 4


(RolloutWorker pid=15280) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.12976800000127753
(RolloutWorker pid=15280)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1
(RolloutWorker pid=15280) reset triggered
(RolloutWorker pid=15280) reload save for track : 4


(RolloutWorker pid=15280) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.12279960000128654
(RolloutWorker pid=15280)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  2
Loop:  3
(RolloutWorker pid=15280) reset triggered
(RolloutWorker pid=15280) reload save for track : 4


...

(RolloutWorker pid=15280) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.12993530032690614
(RolloutWorker pid=15280)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1679
(RolloutWorker pid=15280) reset triggered
(RolloutWorker pid=15280) reload save for track : 4


(RolloutWorker pid=15280) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.140720400326245
(RolloutWorker pid=15280)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
NaN or Inf found in input tensor.
2023-05-24 15:47:04,924	ERROR actor_manager.py:507 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=15280, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000203096B31C0>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File

Loop:  1680
Saved 1680


RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=15280, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000203096B31C0>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
 

In [ ]:
if not debugAsGym and not testResult:
    N = 2000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 50 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()

In [ ]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
#my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [ ]:
if not debugAsGym and testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) 

In [ ]:
if not debugAsGym and testResult:
    from ray.rllib.algorithms.algorithm import Algorithm
    algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_000061")
    #algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_002000")

In [8]:
if not debugAsGym and testResult:
    result = algo.train()

In [ ]:
if not debugAsGym and testResult:

    policy = algo.get_policy()
    #print(policy.model)
    model = policy.model
    print(model)